<a href="https://colab.research.google.com/github/AnnweshaAdhikari/IMDB-review-sentiment-analysis/blob/main/IMDB_review_sentiment_analysis_using_Distilbert_and_CometML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet_ml torch datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.4/677.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 19.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.

In [ ]:
#!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.0 MB/s eta 0:00:00


Setting up Comet_ML

In [ ]:
import comet_ml
comet_ml.init(project_name = "imdb-distilbart")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


Initializing pre-trained distillbert model

In [ ]:
pre_model = "distilbert-base-uncased"
seed = 20

Loading IMDB dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments

dataset_raw = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pre_model)
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset_raw.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Sampling 500 examples from the tokenized IMDB dataset

In [ ]:
training_dataset = tokenized_datasets["train"].shuffle(seed).select(range(200))
testing_dataset = tokenized_datasets["test"].shuffle(seed).select(range(200))

Setting up the transformer model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(pre_model, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation function

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def get_example(index):
  return testing_dataset[index]["text"]

def compute_metrics(pred):
  experiment = comet_ml.get_global_experiment()

  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
  accuracy = accuracy_score(labels, preds)

  if experiment:
    epoch = int(experiment.curr_epoch) if experiment.curr_epoch is not None else 0
    experiment.set_epoch(epoch)
    experiment.log_confusion_matrix(
        y_true = labels,
        y_predicted = preds,
        file_name = f"confusion-matrix-epoch-{epoch}.json",
        labels = ["negative", "positive"],
        index_to_example_function = get_example
    )
    for i in range(20):
     experiment.log_text(get_example(i), metadata={"label": labels[i].item()})

  return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1 score": f1}


Training

In [ ]:
%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

training_args = TrainingArguments(
    seed=seed,
    output_dir = "./results",
    overwrite_output_dir = True,
    num_train_epochs = 1,
    do_train = True,
    do_eval = True,
    evaluation_strategy = "steps",
    eval_steps = 25,
    save_strategy = "steps",
    save_total_limit = 10,
    save_steps = 25,
    per_device_train_batch_size = 8,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset = training_dataset,
    eval_dataset = testing_dataset,
    compute_metrics = compute_metrics,
    data_collator = data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annweshaadhikari/imdb-distilbart/65e28b3b0cfd48bbb0580d169e43f5af

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
25,No log,0.632080,0.760000,0.794809,0.754679,0.750000


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : petite_tongue_3069
COMET INFO:     url                   : https://www.comet.com/annweshaadhikari/imdb-distilbart/65e28b3b0cfd48bbb0580d169e43f5af
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch                          : 1.0
COMET INFO:     eval/accuracy                  : 0.76
COMET INFO:     eval/f1 score                  : 0.75
COMET INFO:     eval/loss                      : 0.6320803761482239
COMET INFO:     eval/precision                 : 0.7948094079480941
COMET INFO:     eval/recall                    : 0.7546792112901611
COMET INFO:     eval/runtime                   : 3.2077
COMET INFO:     eval/samples_per_second        :

TrainOutput(global_step=25, training_loss=0.6696302795410156, metrics={'train_runtime': 19.2167, 'train_samples_per_second': 10.408, 'train_steps_per_second': 1.301, 'total_flos': 26493479731200.0, 'train_loss': 0.6696302795410156, 'epoch': 1.0})